In [ ]:
import os
import sys
import math
from numbers import Number
import unittest
from math import pi
import numpy as np
import logging
from itertools import permutations
from ddt import ddt, data
from numpy.testing import assert_allclose
import contextlib

from qiskit import transpile
from qiskit.exceptions import QiskitError
from qiskit.circuit import QuantumCircuit, QuantumRegister
from qiskit.providers.basicaer import QasmSimulatorPy
from qiskit.quantum_info.random import random_unitary, random_statevector, random_pauli
from qiskit.quantum_info.states import Statevector
from qiskit.circuit.library import QuantumVolume
from qiskit.providers.aer import AerSimulator
from qiskit.quantum_info.operators.operator import Operator
from qiskit.quantum_info.operators.symplectic import Pauli, SparsePauliOp
from qiskit.quantum_info.operators.predicates import matrix_equal
from qiskit.visualization.state_visualization import numbers_to_latex_terms, state_to_latex
from qiskit.circuit.library import QFT, HGate
from test.terra import common
from qiskit_aer.aererror import AerError
from qiskit_aer.quantum_info.states import AerStatevector

In [ ]:
def rand_vec(n, normalize=False):
    """Return complex vector or statevector"""
    seed = np.random.randint(0, np.iinfo(np.int32).max)
    rng = np.random.default_rng(seed)
    vec = rng.random(n) + 1j * rng.random(n)
    if normalize:
        vec /= np.sqrt(np.dot(vec, np.conj(vec)))
    return vec

def assertEqual(a, b):
    assert a == b

def assertNotEqual (a, b):   
    assert a != b

def assertTrue(a):
    assertEqual(a, True)

def assertFalse(a):
    assertEqual(a, False)

def assertIsNone(a):
    assertTrue(a is None)

def assertIn(a, b):
    assertTrue(a in b)

def assertNotIn(a, b):
    assertFalse(a in b)    

def assertLess(a, b):
    assert a < b

def assertRaises(exception, func, *args, **kwds):
    try:
        func(*args, **kwds)
    except exception:
        pass

def assertAlmostEqual(a, b):
    assertTrue(np.isclose(a, b))

def _assertListEqual(list1, list2):
    if len(list1) != len(list2):
        return False
    for elem1, elem2 in zip(list1, list2):
        if list1 != list2:
            return False
    return True

def assertListEqual(list1, list2):
    assertTrue(_assertListEqual(list1, list2))

# https://stackoverflow.com/questions/23549419/assert-that-two-dictionaries-are-almost-equal
def _assertDictAlmostEqual(dict1, dict2, rel_tol=1e-8):
    """
    If dictionary value is a number, then check that the numbers are almost equal, otherwise check if values are exactly equal
    Note: does not currently try converting strings to digits and comparing them. Does not care about ordering of keys in dictionaries
    Just returns true or false
    """
    if len(dict1) != len(dict2):
        return False
    # Loop through each item in the first dict and compare it to the second dict
    for key, item in dict1.items():
        # If it is a nested dictionary, need to call the function again
        if isinstance(item, dict):
            # If the nested dictionaries are not almost equal, return False
            if not _assertDictAlmostEqual(dict1[key], dict2[key], rel_tol=rel_tol):
                return False
        # If it's not a dictionary, then continue comparing
        # Put in else statement or else the nested dictionary will get compared twice and
        # On the second time will check for exactly equal and will fail
        else:
            # If the value is a number, check if they are approximately equal
            if isinstance(item, Number):
                # if not abs(dict1[key] - dict2[key]) <= rel_tol:
                # https://stackoverflow.com/questions/5595425/what-is-the-best-way-to-compare-floats-for-almost-equality-in-python
                if hasattr(item, 'imag'):
                    if not math.isclose(dict1[key].real, dict2[key].real, rel_tol=rel_tol) or not math.isclose(dict1[key].imag, dict2[key].imag, rel_tol=rel_tol):
                        return False
                else:
                    if not math.isclose(dict1[key], dict2[key], rel_tol=rel_tol):
                        return False
            else:
                if not (dict1[key] == dict2[key]):
                    return False
    return True

def assertDictAlmostEqual(dict1, dict2, rel_tol=1e-8):
    assertTrue(_assertDictAlmostEqual(dict1, dict2, rel_tol))

@contextlib.contextmanager
def subTest(**kwds):
    try:
        yield
    except Exception as e:
        msg = kwds['msg'] if 'msg' in kwds else ''
        print(e, msg, file=sys.stderr)

In [ ]:
"""Test generation of Aer's StateVector with QV """
circ = QuantumVolume(5, seed=1111)
state = AerStatevector(circ)
expected = Statevector(circ)

for e, s in zip(expected, state):
    assertAlmostEqual(e, s)

In [ ]:
"""Test method and device properties"""
circ = QuantumVolume(5, seed=1111)
state1 = AerStatevector(circ)

assertEqual('statevector', state1.metadata()['method'])
assertEqual('CPU', state1.metadata()['device'])

state2 = AerStatevector(circ, method='matrix_product_state')
assertEqual('matrix_product_state', state2.metadata()['method'])
assertEqual('CPU', state2.metadata()['device'])

assertEqual(state1, state2)

In [ ]:
"""Test method and device properties"""
circ1 = QuantumVolume(5, seed=1111)
circ2 = circ1.compose(circ1)
circ3 = circ2.compose(circ1)

state1 = AerStatevector(circ1)
state2 = AerStatevector(circ2)
state3 = AerStatevector(circ3)

assertEqual(state1.evolve(circ1), state2)
assertEqual(state1.evolve(circ1).evolve(circ1), state3)

In [ ]:
"""Test basic gates can be decomposed correctly"""
circ = QuantumCircuit(3)
circ.h(0)
circ.x(1)
circ.ry(np.pi / 2, 2)
state1 = AerStatevector(circ)

In [ ]:
# Test tensor product of 1-qubit gates
circuit = QuantumCircuit(3)
circuit.h(0)
circuit.x(1)
circuit.ry(np.pi / 2, 2)
psi = AerStatevector.from_instruction(circuit)
target = AerStatevector.from_label("000").evolve(Operator(circuit))
assertEqual(target, psi)

In [ ]:
# Test tensor product of 1-qubit gates
circuit = QuantumCircuit(3)
circuit.h(0)
circuit.h(1)
target = AerStatevector.from_label("000").evolve(Operator(circuit))
psi = AerStatevector.from_instruction(circuit)
assertEqual(psi, target)

In [ ]:
"""Test deep copy"""
import copy
circ1 = QuantumVolume(5, seed=1111)

state1 = AerStatevector(circ1)
state2 = copy.deepcopy(state1)

for pa1, pa2 in zip(state1, state2):
    assertAlmostEqual(pa1, pa2)

assertNotEqual(id(state1._data), id(state2._data))

In [ ]:
"""Test ndarray initialization """
circ = QuantumVolume(5, seed=1111)
expected = Statevector(circ)
state = AerStatevector(expected.data)

for e, s in zip(expected, state):
    assertAlmostEqual(e, s)

In [ ]:
"""Test Statevector initialization """
circ = QuantumVolume(5, seed=1111)
expected = Statevector(circ)
state = AerStatevector(expected)

for e, s in zip(expected, state):
    assertAlmostEqual(e, s)

## Copy from test_statevector.py in terra

In [ ]:
"""Test subsystem initialization from N-qubit array."""
# Test automatic inference of qubit subsystems
vec = rand_vec(8)
for dims in [None, 8]:
    state = AerStatevector(vec, dims=dims)
    assert_allclose(state.data, vec)
    assertEqual(state.dim, 8)
    assertEqual(state.dims(), (2, 2, 2))
    assertEqual(state.num_qubits, 3)

In [ ]:
"""Test subsystem initialization from N-qubit array."""
# Test automatic inference of qubit subsystems
vec = rand_vec(8)
for dims in [None, 8]:
    state = AerStatevector(vec, dims=dims)
    assert_allclose(state.data, vec)
    assertEqual(state.dim, 8)
    assertEqual(state.dims(), (2, 2, 2))
    assertEqual(state.num_qubits, 3)

In [ ]:
"""Test initialization from circuit."""
circuit = QuantumCircuit(3)
circuit.x(0)
state = AerStatevector(circuit)

assertEqual(state.dim, 8)
assertEqual(state.dims(), (2, 2, 2))
assertTrue(all(state.data == np.array([0, 1, 0, 0, 0, 0, 0, 0], dtype=complex)))
assertEqual(state.num_qubits, 3)

In [ ]:
"""Test initialization exception from array."""
vec = rand_vec(4)
assertRaises(QiskitError, AerStatevector, vec, dims=[4, 2])
assertRaises(QiskitError, AerStatevector, vec, dims=[2, 4])
assertRaises(QiskitError, AerStatevector, vec, dims=5)

In [ ]:
"""Test initialization from AerStatevector."""
vec1 = AerStatevector(rand_vec(4))
vec2 = AerStatevector(vec1)
assertEqual(vec1, vec2)

In [ ]:
"""Test initialization from a circuit."""
# random unitaries
u0 = random_unitary(2).data
u1 = random_unitary(2).data
# add to circuit
qr = QuantumRegister(2)
circ = QuantumCircuit(qr)
circ.unitary(u0, [qr[0]])
circ.unitary(u1, [qr[1]])
target = AerStatevector(np.kron(u1, u0).dot([1, 0, 0, 0]))
vec = AerStatevector.from_instruction(circ)
assertEqual(vec, target)

# Test tensor product of 1-qubit gates
circuit = QuantumCircuit(3)
circuit.h(0)
circuit.x(1)
circuit.ry(np.pi / 2, 2)
target = AerStatevector.from_label("000").evolve(Operator(circuit))
psi = AerStatevector.from_instruction(circuit)
assertEqual(psi, target)

# Test decomposition of Controlled-Phase gate
lam = np.pi / 4
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.h(1)
circuit.cp(lam, 0, 1)
target = AerStatevector.from_label("00").evolve(Operator(circuit))
psi = AerStatevector.from_instruction(circuit)
assertEqual(psi, target)

# Test decomposition of controlled-H gate
circuit = QuantumCircuit(2)
circ.x(0)
circuit.ch(0, 1)
target = AerStatevector.from_label("00").evolve(Operator(circuit))
psi = AerStatevector.from_instruction(circuit)
assertEqual(psi, target)

# Test custom controlled gate
qc = QuantumCircuit(2)
qc.x(0)
qc.h(1)
gate = qc.to_gate()
gate_ctrl = gate.control()

circuit = QuantumCircuit(3)
circuit.x(0)
circuit.append(gate_ctrl, range(3))
target = AerStatevector.from_label("000").evolve(Operator(circuit))
psi = AerStatevector.from_instruction(circuit)
assertEqual(psi, target)

# Test initialize instruction
target = AerStatevector([1, 0, 0, 1j]) / np.sqrt(2)
circuit = QuantumCircuit(2)
circuit.initialize(target.data, [0, 1])
psi = AerStatevector.from_instruction(circuit)
assertEqual(psi, target)

# Test reset instruction
target = AerStatevector([1, 0])
circuit = QuantumCircuit(1)
circuit.h(0)
circuit.reset(0)
psi = AerStatevector.from_instruction(circuit)
assertEqual(psi, target)

In [ ]:
"""Test initialization from an instruction."""
target = np.dot(HGate().to_matrix(), [1, 0])
vec = AerStatevector.from_instruction(HGate()).data
global_phase_equivalent = matrix_equal(vec, target, ignore_phase=True)
assertTrue(global_phase_equivalent)

In [ ]:
"""Test initialization from a label"""
x_p = AerStatevector(np.array([1, 1]) / np.sqrt(2))
x_m = AerStatevector(np.array([1, -1]) / np.sqrt(2))
y_p = AerStatevector(np.array([1, 1j]) / np.sqrt(2))
y_m = AerStatevector(np.array([1, -1j]) / np.sqrt(2))
z_p = AerStatevector(np.array([1, 0]))
z_m = AerStatevector(np.array([0, 1]))

label = "01"
target = z_p.tensor(z_m)
assertEqual(target, AerStatevector.from_label(label))

label = "+-"
target = x_p.tensor(x_m)
assertEqual(target, AerStatevector.from_label(label))

label = "rl"
target = y_p.tensor(y_m)
assertEqual(target, AerStatevector.from_label(label))

In [ ]:
"""Test __eq__ method"""
for _ in range(10):
    vec = rand_vec(4)
    assertEqual(AerStatevector(vec), AerStatevector(vec.tolist()))

In [ ]:
"""Test __getitem__ method"""
for _ in range(10):
    vec = rand_vec(4)
    state = AerStatevector(vec)
    for i in range(4):
        assertEqual(state[i], vec[i])
        assertEqual(state[format(i, "b")], vec[i])

In [ ]:
"""Test __getitem__ method raises exceptions."""
for i in range(1, 4):
    state = AerStatevector(rand_vec(2**i))
    assertRaises(QiskitError, state.__getitem__, 2**i)
    assertRaises(QiskitError, state.__getitem__, -1)

In [ ]:
"""Test AerStatevector copy method"""
for _ in range(5):
    vec = rand_vec(4)
    orig = AerStatevector(vec)
    cpy = orig.copy()
    cpy._data[0] += 1.0
    assertFalse(cpy == orig)

In [ ]:
"""Test is_valid method."""
state = AerStatevector([1, 1])
assertFalse(state.is_valid())
for _ in range(10):
    state = AerStatevector(rand_vec(4, normalize=True))
    assertTrue(state.is_valid())

In [ ]:
"""Test to_operator method for returning projector."""
for _ in range(10):
    vec = rand_vec(4)
    target = Operator(np.outer(vec, np.conj(vec)))
    op = AerStatevector(vec).to_operator()
    assertEqual(op, target)

In [ ]:
"""Test _evolve method."""
for _ in range(10):
    op = random_unitary(4)
    vec = rand_vec(4)
    target = AerStatevector(np.dot(op.data, vec))
    evolved = AerStatevector(vec).evolve(op)
    assertEqual(target, evolved)

In [ ]:
"""Test subsystem _evolve method."""
# Test evolving single-qubit of 3-qubit system
for _ in range(5):
    vec = rand_vec(8)
    state = AerStatevector(vec)
    op0 = random_unitary(2)
    op1 = random_unitary(2)
    op2 = random_unitary(2)

    # Test evolve on 1-qubit
    op = op0
    op_full = Operator(np.eye(4)).tensor(op)
    target = AerStatevector(np.dot(op_full.data, vec))
    assertEqual(state.evolve(op, qargs=[0]), target)

    # Evolve on qubit 1
    op_full = Operator(np.eye(2)).tensor(op).tensor(np.eye(2))
    target = AerStatevector(np.dot(op_full.data, vec))
    assertEqual(state.evolve(op, qargs=[1]), target)

    # Evolve on qubit 2
    op_full = op.tensor(np.eye(4))
    target = AerStatevector(np.dot(op_full.data, vec))
    assertEqual(state.evolve(op, qargs=[2]), target)

    # Test evolve on 2-qubits
    op = op1.tensor(op0)

    # Evolve on qubits [0, 2]
    op_full = op1.tensor(np.eye(2)).tensor(op0)
    target = AerStatevector(np.dot(op_full.data, vec))
    assertEqual(state.evolve(op, qargs=[0, 2]), target)

    # Evolve on qubits [2, 0]
    op_full = op0.tensor(np.eye(2)).tensor(op1)
    target = AerStatevector(np.dot(op_full.data, vec))
    assertEqual(state.evolve(op, qargs=[2, 0]), target)

    # Test evolve on 3-qubits
    op = op2.tensor(op1).tensor(op0)

    # Evolve on qubits [0, 1, 2]
    op_full = op
    target = AerStatevector(np.dot(op_full.data, vec))
    assertEqual(state.evolve(op, qargs=[0, 1, 2]), target)

    # Evolve on qubits [2, 1, 0]
    op_full = op0.tensor(op1).tensor(op2)
    target = AerStatevector(np.dot(op_full.data, vec))
    assertEqual(state.evolve(op, qargs=[2, 1, 0]), target)

In [ ]:
"""Test evolve circuit with global phase."""
state_i = AerStatevector([1, 0])
qr = QuantumRegister(2)
phase = np.pi / 4
circ = QuantumCircuit(qr, global_phase=phase)
circ.x(0)
state_f = state_i.evolve(circ, qargs=[0])
target = AerStatevector([0, 1]) * np.exp(1j * phase)

assertEqual(state_f, target)

In [ ]:
"""Test conjugate method."""
for _ in range(10):
    vec = rand_vec(4)
    target = AerStatevector(np.conj(vec))
    state = AerStatevector(vec).conjugate()
    assertEqual(state, target)

In [ ]:
"""Test expand method."""
for _ in range(10):
    vec0 = rand_vec(2)
    vec1 = rand_vec(4)
    target = np.kron(vec1, vec0)
    state = AerStatevector(vec0).expand(AerStatevector(vec1))
    assertEqual(state.dim, 8)
    assertEqual(state.dims(), (2, 2, 2))
    assert_allclose(state.data, target)

In [ ]:
"""Test tensor method."""
for _ in range(10):
    vec0 = rand_vec(2)
    vec1 = rand_vec(4)
    target = np.kron(vec0, vec1)
    state = AerStatevector(vec0).tensor(AerStatevector(vec1))
    assertEqual(state.dim, 8)
    assertEqual(state.dims(), (2, 2, 2))
    assert_allclose(state.data, target)

In [ ]:
"""Test inner method."""
for _ in range(10):
    vec0 = AerStatevector(rand_vec(4))
    vec1 = AerStatevector(rand_vec(4))
    target = np.vdot(vec0.data, vec1.data)
    result = vec0.inner(vec1)
    assertAlmostEqual(result, target)

In [ ]:
"""Test add method."""
for _ in range(10):
    vec0 = rand_vec(4)
    vec1 = rand_vec(4)
    state0 = AerStatevector(vec0)
    state1 = AerStatevector(vec1)
    assertEqual(state0 + state1, AerStatevector(vec0 + vec1))

In [ ]:
"""Test add method raises exceptions."""
state1 = AerStatevector(rand_vec(2))
state2 = AerStatevector(rand_vec(4))
assertRaises(QiskitError, state1.__add__, state2)

In [ ]:
"""Test subtract method."""
for _ in range(10):
    vec0 = rand_vec(4)
    vec1 = rand_vec(4)
    state0 = AerStatevector(vec0)
    state1 = AerStatevector(vec1)
    assertEqual(state0 - state1, AerStatevector(vec0 - vec1))

In [ ]:
"""Test multiply method."""
for _ in range(10):
    vec = rand_vec(4)
    state = AerStatevector(vec)
    val = np.random.rand() + 1j * np.random.rand()
    assertEqual(val * state, AerStatevector(val * state))

In [ ]:
"""Test negate method"""
for _ in range(10):
    vec = rand_vec(4)
    state = AerStatevector(vec)
    assertEqual(-state, AerStatevector(-1 * vec))

In [ ]:
"""Test equiv method"""
vec = np.array([1, 0, 0, -1j]) / np.sqrt(2)
phase = np.exp(-1j * np.pi / 4)
statevec = AerStatevector(vec)
assertTrue(statevec.equiv(phase * vec))
assertTrue(statevec.equiv(AerStatevector(phase * vec)))
assertFalse(statevec.equiv(2 * vec))

In [ ]:
"""Test the equiv method on different types of input."""
statevec = AerStatevector([1, 0])

qc = QuantumCircuit(1)
assertTrue(statevec.equiv(qc))
qc.x(0)
assertFalse(statevec.equiv(qc))

In [ ]:
"""Test to_dict method"""

with subTest(msg="dims = (2, 3)"):
    assertRaises(Exception, AerStatevector, np.arange(1, 7), dims=(2, 3))

with subTest(msg="dims = (11, )"):
    assertRaises(Exception, AerStatevector, np.arange(1, 12), dims=(11,))

with subTest(msg="dims = (2, 11)"):
    assertRaises(Exception, AerStatevector, np.arange(1, 23), dims=(2, 11))

In [ ]:
"""Test probabilities method for product state"""

state = AerStatevector.from_label("+0")

# 2-qubit qargs
with subTest(msg="P(None)"):
    probs = state.probabilities()
    target = np.array([0.5, 0, 0.5, 0])
    assertTrue(np.allclose(probs, target))

with subTest(msg="P([0, 1])"):
    probs = state.probabilities([0, 1])
    target = np.array([0.5, 0, 0.5, 0])
    assertTrue(np.allclose(probs, target))

with subTest(msg="P([1, 0]"):
    probs = state.probabilities([1, 0])
    target = np.array([0.5, 0.5, 0, 0])
    assertTrue(np.allclose(probs, target))

# 1-qubit qargs
with subTest(msg="P([0])"):
    probs = state.probabilities([0])
    target = np.array([1, 0])
    assertTrue(np.allclose(probs, target))

with subTest(msg="P([1])"):
    probs = state.probabilities([1])
    target = np.array([0.5, 0.5])
    assertTrue(np.allclose(probs, target))

In [ ]:
"""Test probabilities method for GHZ state"""

state = (AerStatevector.from_label("000") + AerStatevector.from_label("111")) / np.sqrt(2)

# 3-qubit qargs
target = np.array([0.5, 0, 0, 0, 0, 0, 0, 0.5])
for qargs in [[0, 1, 2], [2, 1, 0], [1, 2, 0], [1, 0, 2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities(qargs)
        assertTrue(np.allclose(probs, target))

# 2-qubit qargs
target = np.array([0.5, 0, 0, 0.5])
for qargs in [[0, 1], [2, 1], [1, 2], [1, 2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities(qargs)
        assertTrue(np.allclose(probs, target))

# 1-qubit qargs
target = np.array([0.5, 0.5])
for qargs in [[0], [1], [2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities(qargs)
        assertTrue(np.allclose(probs, target))

In [ ]:
"""Test probabilities method with W state"""

state = (
    AerStatevector.from_label("001")
    + AerStatevector.from_label("010")
    + AerStatevector.from_label("100")
) / np.sqrt(3)

# 3-qubit qargs
target = np.array([0, 1 / 3, 1 / 3, 0, 1 / 3, 0, 0, 0])
for qargs in [[0, 1, 2], [2, 1, 0], [1, 2, 0], [1, 0, 2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities(qargs)
        assertTrue(np.allclose(probs, target))

# 2-qubit qargs
target = np.array([1 / 3, 1 / 3, 1 / 3, 0])
for qargs in [[0, 1], [2, 1], [1, 2], [1, 2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities(qargs)
        assertTrue(np.allclose(probs, target))

# 1-qubit qargs
target = np.array([2 / 3, 1 / 3])
for qargs in [[0], [1], [2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities(qargs)
        assertTrue(np.allclose(probs, target))

In [ ]:
"""Test probabilities_dict method for product state"""

state = AerStatevector.from_label("+0")

# 2-qubit qargs
with subTest(msg="P(None)"):
    probs = state.probabilities_dict()
    target = {"00": 0.5, "10": 0.5}
    assertDictAlmostEqual(probs, target)

with subTest(msg="P([0, 1])"):
    probs = state.probabilities_dict([0, 1])
    target = {"00": 0.5, "10": 0.5}
    assertDictAlmostEqual(probs, target)

with subTest(msg="P([1, 0]"):
    probs = state.probabilities_dict([1, 0])
    target = {"00": 0.5, "01": 0.5}
    assertDictAlmostEqual(probs, target)

# 1-qubit qargs
with subTest(msg="P([0])"):
    probs = state.probabilities_dict([0])
    target = {"0": 1}
    assertDictAlmostEqual(probs, target)

with subTest(msg="P([1])"):
    probs = state.probabilities_dict([1])
    target = {"0": 0.5, "1": 0.5}
    assertDictAlmostEqual(probs, target)

In [ ]:
"""Test probabilities_dict method for GHZ state"""

state = (AerStatevector.from_label("000") + AerStatevector.from_label("111")) / np.sqrt(2)

# 3-qubit qargs
target = {"000": 0.5, "111": 0.5}
for qargs in [[0, 1, 2], [2, 1, 0], [1, 2, 0], [1, 0, 2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities_dict(qargs)
        assertDictAlmostEqual(probs, target)

# 2-qubit qargs
target = {"00": 0.5, "11": 0.5}
for qargs in [[0, 1], [2, 1], [1, 2], [1, 2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities_dict(qargs)
        assertDictAlmostEqual(probs, target)

# 1-qubit qargs
target = {"0": 0.5, "1": 0.5}
for qargs in [[0], [1], [2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities_dict(qargs)
        assertDictAlmostEqual(probs, target)

In [ ]:
"""Test probabilities_dict method with W state"""

state = (
    AerStatevector.from_label("001")
    + AerStatevector.from_label("010")
    + AerStatevector.from_label("100")
) / np.sqrt(3)

# 3-qubit qargs
target = np.array([0, 1 / 3, 1 / 3, 0, 1 / 3, 0, 0, 0])
target = {"001": 1 / 3, "010": 1 / 3, "100": 1 / 3}
for qargs in [[0, 1, 2], [2, 1, 0], [1, 2, 0], [1, 0, 2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities_dict(qargs)
        assertDictAlmostEqual(probs, target)

# 2-qubit qargs
target = {"00": 1 / 3, "01": 1 / 3, "10": 1 / 3}
for qargs in [[0, 1], [2, 1], [1, 2], [1, 2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities_dict(qargs)
        assertDictAlmostEqual(probs, target)

# 1-qubit qargs
target = {"0": 2 / 3, "1": 1 / 3}
for qargs in [[0], [1], [2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities_dict(qargs)
        assertDictAlmostEqual(probs, target)

In [ ]:
"""Test sample_counts method for GHZ state"""

shots = 2000
threshold = 0.02 * shots
state = (AerStatevector.from_label("000") + AerStatevector.from_label("111")) / np.sqrt(2)
state.seed(100)

# 3-qubit qargs
target = {"000": shots / 2, "111": shots / 2}
for qargs in [[0, 1, 2], [2, 1, 0], [1, 2, 0], [1, 0, 2]]:

    with subTest(msg=f"counts (qargs={qargs})"):
        counts = state.sample_counts(shots, qargs=qargs)
        assertDictAlmostEqual(counts, target, threshold)

# 2-qubit qargs
target = {"00": shots / 2, "11": shots / 2}
for qargs in [[0, 1], [2, 1], [1, 2], [1, 2]]:

    with subTest(msg=f"counts (qargs={qargs})"):
        counts = state.sample_counts(shots, qargs=qargs)
        assertDictAlmostEqual(counts, target, threshold)

# 1-qubit qargs
target = {"0": shots / 2, "1": shots / 2}
for qargs in [[0], [1], [2]]:

    with subTest(msg=f"counts (qargs={qargs})"):
        counts = state.sample_counts(shots, qargs=qargs)
        assertDictAlmostEqual(counts, target, threshold)

In [ ]:
"""Test sample_counts method for W state"""
shots = 3000
threshold = 0.02 * shots
state = (
    AerStatevector.from_label("001")
    + AerStatevector.from_label("010")
    + AerStatevector.from_label("100")
) / np.sqrt(3)
state.seed(100)

target = {"001": shots / 3, "010": shots / 3, "100": shots / 3}
for qargs in [[0, 1, 2], [2, 1, 0], [1, 2, 0], [1, 0, 2]]:

    with subTest(msg=f"P({qargs})"):
        counts = state.sample_counts(shots, qargs=qargs)
        assertDictAlmostEqual(counts, target, threshold)

# 2-qubit qargs
target = {"00": shots / 3, "01": shots / 3, "10": shots / 3}
for qargs in [[0, 1], [2, 1], [1, 2], [1, 2]]:

    with subTest(msg=f"P({qargs})"):
        counts = state.sample_counts(shots, qargs=qargs)
        assertDictAlmostEqual(counts, target, threshold)

# 1-qubit qargs
target = {"0": 2 * shots / 3, "1": shots / 3}
for qargs in [[0], [1], [2]]:

    with subTest(msg=f"P({qargs})"):
        counts = state.sample_counts(shots, qargs=qargs)
        assertDictAlmostEqual(counts, target, threshold)

In [ ]:
"""Test sample_memory method for GHZ state"""

shots = 2000
state = (AerStatevector.from_label("000") + AerStatevector.from_label("111")) / np.sqrt(2)
state.seed(100)

# 3-qubit qargs
target = {"000": shots / 2, "111": shots / 2}
for qargs in [[0, 1, 2], [2, 1, 0], [1, 2, 0], [1, 0, 2]]:

    with subTest(msg=f"memory (qargs={qargs})"):
        memory = state.sample_memory(shots, qargs=qargs)
        assertEqual(len(memory), shots)
        assertEqual(set(memory), set(target))

# 2-qubit qargs
target = {"00": shots / 2, "11": shots / 2}
for qargs in [[0, 1], [2, 1], [1, 2], [1, 2]]:

    with subTest(msg=f"memory (qargs={qargs})"):
        memory = state.sample_memory(shots, qargs=qargs)
        assertEqual(len(memory), shots)
        assertEqual(set(memory), set(target))

# 1-qubit qargs
target = {"0": shots / 2, "1": shots / 2}
for qargs in [[0], [1], [2]]:

    with subTest(msg=f"memory (qargs={qargs})"):
        memory = state.sample_memory(shots, qargs=qargs)
        assertEqual(len(memory), shots)
        assertEqual(set(memory), set(target))

In [ ]:
"""Test sample_memory method for W state"""
shots = 3000
state = (
    AerStatevector.from_label("001")
    + AerStatevector.from_label("010")
    + AerStatevector.from_label("100")
) / np.sqrt(3)
state.seed(100)

target = {"001": shots / 3, "010": shots / 3, "100": shots / 3}
for qargs in [[0, 1, 2], [2, 1, 0], [1, 2, 0], [1, 0, 2]]:

    with subTest(msg=f"memory (qargs={qargs})"):
        memory = state.sample_memory(shots, qargs=qargs)
        assertEqual(len(memory), shots)
        assertEqual(set(memory), set(target))

# 2-qubit qargs
target = {"00": shots / 3, "01": shots / 3, "10": shots / 3}
for qargs in [[0, 1], [2, 1], [1, 2], [1, 2]]:

    with subTest(msg=f"memory (qargs={qargs})"):
        memory = state.sample_memory(shots, qargs=qargs)
        assertEqual(len(memory), shots)
        assertEqual(set(memory), set(target))

# 1-qubit qargs
target = {"0": 2 * shots / 3, "1": shots / 3}
for qargs in [[0], [1], [2]]:

    with subTest(msg=f"memory (qargs={qargs})"):
        memory = state.sample_memory(shots, qargs=qargs)
        assertEqual(len(memory), shots)
        assertEqual(set(memory), set(target))

In [ ]:
"""Test reset method for 2-qubit state"""

state = AerStatevector(np.array([1, 0, 0, 1]) / np.sqrt(2))
state.seed(100)

with subTest(msg="reset"):
    psi = state.copy()
    value = psi.reset()
    target = AerStatevector(np.array([1, 0, 0, 0]))
    assertEqual(value, target)

with subTest(msg="reset"):
    psi = state.copy()
    value = psi.reset([0, 1])
    target = AerStatevector(np.array([1, 0, 0, 0]))
    assertEqual(value, target)

with subTest(msg="reset [0]"):
    psi = state.copy()
    value = psi.reset([0])
    targets = [AerStatevector(np.array([1, 0, 0, 0])), AerStatevector(np.array([0, 0, 1, 0]))]
    assertIn(value, targets)

with subTest(msg="reset [0]"):
    psi = state.copy()
    value = psi.reset([1])
    targets = [AerStatevector(np.array([1, 0, 0, 0])), AerStatevector(np.array([0, 1, 0, 0]))]
    assertIn(value, targets)

In [ ]:
"""Test measure method for 2-qubit state"""

state = AerStatevector.from_label("+0")
seed = 200
shots = 100

with subTest(msg="measure"):
    for i in range(shots):
        psi = state.copy()
        psi.seed(seed + i)
        outcome, value = psi.measure()
        assertIn(outcome, ["00", "10"])
        if outcome == "00":
            target = AerStatevector.from_label("00")
            assertEqual(value, target)
        else:
            target = AerStatevector.from_label("10")
            assertEqual(value, target)

with subTest(msg="measure [0, 1]"):
    for i in range(shots):
        psi = state.copy()
        outcome, value = psi.measure([0, 1])
        assertIn(outcome, ["00", "10"])
        if outcome == "00":
            target = AerStatevector.from_label("00")
            assertEqual(value, target)
        else:
            target = AerStatevector.from_label("10")
            assertEqual(value, target)

with subTest(msg="measure [1, 0]"):
    for i in range(shots):
        psi = state.copy()
        outcome, value = psi.measure([1, 0])
        assertIn(outcome, ["00", "01"])
        if outcome == "00":
            target = AerStatevector.from_label("00")
            assertEqual(value, target)
        else:
            target = AerStatevector.from_label("10")
            assertEqual(value, target)

with subTest(msg="measure [0]"):
    for i in range(shots):
        psi = state.copy()
        outcome, value = psi.measure([0])
        assertEqual(outcome, "0")
        target = AerStatevector(np.array([1, 0, 1, 0]) / np.sqrt(2))
        assertEqual(value, target)

with subTest(msg="measure [1]"):
    for i in range(shots):
        psi = state.copy()
        outcome, value = psi.measure([1])
        assertIn(outcome, ["0", "1"])
        if outcome == "0":
            target = AerStatevector.from_label("00")
            assertEqual(value, target)
        else:
            target = AerStatevector.from_label("10")
            assertEqual(value, target)

In [ ]:
"""Test from_int method"""

with subTest(msg="from_int(0, 4)"):
    target = AerStatevector([1, 0, 0, 0])
    value = AerStatevector.from_int(0, 4)
    assertEqual(target, value)

with subTest(msg="from_int(3, 4)"):
    target = AerStatevector([0, 0, 0, 1])
    value = AerStatevector.from_int(3, 4)
    assertEqual(target, value)

In [ ]:
"""Test expectation_value method"""

psi = AerStatevector([1, 0, 0, 1]) / np.sqrt(2)
for label, target in [
    ("II", 1),
    ("XX", 1),
    ("YY", -1),
    ("ZZ", 1),
    ("IX", 0),
    ("YZ", 0),
    ("ZX", 0),
    ("YI", 0),
]:
    with subTest(msg=f"<{label}>"):
        op = Pauli(label)
        expval = psi.expectation_value(op)
        assertAlmostEqual(expval, target)

psi = AerStatevector([np.sqrt(2), 0, 0, 0, 0, 0, 0, 1 + 1j]) / 2
for label, target in [
    ("XXX", np.sqrt(2) / 2),
    ("YYY", -np.sqrt(2) / 2),
    ("ZZZ", 0),
    ("XYZ", 0),
    ("YIY", 0),
]:
    with subTest(msg=f"<{label}>"):
        op = Pauli(label)
        expval = psi.expectation_value(op)
        assertAlmostEqual(expval, target)

labels = ["XXX", "IXI", "YYY", "III"]
coeffs = [3.0, 5.5, -1j, 23]
spp_op = SparsePauliOp.from_list(list(zip(labels, coeffs)))
expval = psi.expectation_value(spp_op)
target = 25.121320343559642 + 0.7071067811865476j
assertAlmostEqual(expval, target)

In [ ]:
for pauli in (
    "II",
    "IX",
    "IY",
    "IZ",
    "XI",
    "XX",
    "XY",
    "XZ",
    "YI",
    "YX",
    "YY",
    "YZ",
    "ZI",
    "ZX",
    "ZY",
    "ZZ",
    "-II",
    "-IX",
    "-IY",
    "-IZ",
    "-XI",
    "-XX",
    "-XY",
    "-XZ",
    "-YI",
    "-YX",
    "-YY",
    "-YZ",
    "-ZI",
    "-ZX",
    "-ZY",
    "-ZZ",
    "iII",
    "iIX",
    "iIY",
    "iIZ",
    "iXI",
    "iXX",
    "iXY",
    "iXZ",
    "iYI",
    "iYX",
    "iYY",
    "iYZ",
    "iZI",
    "iZX",
    "iZY",
    "iZZ",
    "-iII",
    "-iIX",
    "-iIY",
    "-iIZ",
    "-iXI",
    "-iXX",
    "-iXY",
    "-iXZ",
    "-iYI",
    "-iYX",
    "-iYY",
    "-iYZ",
    "-iZI",
    "-iZX",
    "-iZY",
    "-iZZ",
):
    """Test expectation_value method for Pauli op"""
    seed = 1020
    op = Pauli(pauli)
    state = AerStatevector(random_statevector(2**op.num_qubits, seed=seed).data)
    target = state.expectation_value(op.to_matrix())
    expval = state.expectation_value(op)
    assertAlmostEqual(expval, target)

In [ ]:
for qubits in ([0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1]):
    """Test expectation_value method for Pauli op"""
    seed = 1020
    op = random_pauli(2, seed=seed)
    state = AerStatevector(random_statevector(2**3, seed=seed).data)
    target = state.expectation_value(op.to_matrix(), qubits)
    expval = state.expectation_value(op, qubits)
    assertAlmostEqual(expval, target)

In [ ]:
for qargs in (qargs for i in range(4) for qargs in permutations(range(4), r=i + 1)):
    """Test probabilities method with qargs"""
    # Get initial state
    nq = 4
    nc = len(qargs)
    state_circ = QuantumCircuit(nq, nc)
    for i in range(nq):
        state_circ.ry((i + 1) * np.pi / (nq + 1), i)

    # Get probabilities
    state = AerStatevector(state_circ)
    probs = state.probabilities(qargs)

    # Estimate target probs from simulator measurement
    sim = QasmSimulatorPy()
    shots = 5000
    seed = 100
    circ = transpile(state_circ, sim)
    circ.measure(qargs, range(nc))
    result = sim.run(circ, shots=shots, seed_simulator=seed).result()
    target = np.zeros(2**nc, dtype=float)
    for i, p in result.get_counts(0).int_outcomes().items():
        target[i] = p / shots
    # Compare
    delta = np.linalg.norm(probs - target)
    assertLess(delta, 0.05)

In [ ]:
"""Test global phase is handled correctly when evolving statevector."""

qc = QuantumCircuit(1)
qc.rz(0.5, 0)
qc2 = transpile(qc, basis_gates=["p"])
sv = AerStatevector.from_instruction(qc2)
expected = np.array([0.96891242 - 0.24740396j, 0])
assertEqual(float(qc2.global_phase), 2 * np.pi - 0.25)
assertEqual(sv, AerStatevector(expected))

In [ ]:
"""Test reverse_qargs method"""
circ1 = QFT(5)
circ2 = circ1.reverse_bits()

state1 = AerStatevector.from_instruction(circ1)
state2 = AerStatevector.from_instruction(circ2)
assertEqual(state1.reverse_qargs(), state2)

In [ ]:
"""Test draw method"""
qc1 = QFT(5)
sv = AerStatevector.from_instruction(qc1)
with subTest(msg="str(statevector)"):
    str(sv)
for drawtype in ["repr", "text", "latex", "latex_source", "qsphere", "hinton", "bloch"]:
    with subTest(msg=f"draw('{drawtype}')"):
        sv.draw(drawtype)
with subTest(msg=" draw('latex', convention='vector')"):
    sv.draw("latex", convention="vector")

In [ ]:
"""Test conversion of large dense state vector"""
sv = AerStatevector(np.ones((2**15, 1)))
latex_representation = state_to_latex(sv)
assertEqual(
    latex_representation,
    " |000000000000000\\rangle+ |000000000000001\\rangle+ |000000000000010\\rangle+"
    " |000000000000011\\rangle+ |000000000000100\\rangle+ |000000000000101\\rangle +"
    " \\ldots + |111111111111011\\rangle+ |111111111111100\\rangle+"
    " |111111111111101\\rangle+ |111111111111110\\rangle+ |111111111111111\\rangle",
)

In [ ]:
"""Test conversion of large sparse state vector"""
sv = AerStatevector(np.eye(2**15, 1))
latex_representation = state_to_latex(sv)
assertEqual(latex_representation, " |000000000000000\\rangle")

In [ ]:
"""Test conversions of complex numbers to latex terms"""

cases = [
    ([1 - 8e-17, 0], ["", None]),
    ([0, -1], [None, "-"]),
    ([0, 1], [None, ""]),
    ([0, 1j], [None, "i"]),
    ([-1, 1], ["-", "+"]),
    ([0, 1j], [None, "i"]),
    ([-1, 1j], ["-", "+i"]),
    ([1e-16 + 1j], ["i"]),
    ([-1 + 1e-16 * 1j], ["-"]),
    ([-1, -1 - 1j], ["-", "+ (-1 - i)"]),
    ([np.sqrt(2) / 2, np.sqrt(2) / 2], ["\\frac{\\sqrt{2}}{2}", "+\\frac{\\sqrt{2}}{2}"]),
    ([1 + np.sqrt(2)], ["(1 + \\sqrt{2})"]),
]
for numbers, latex_terms in cases:
    terms = numbers_to_latex_terms(numbers)
    assertListEqual(terms, latex_terms)

In [ ]:
"""Test numerical rounding errors are not printed"""
sv = AerStatevector(np.array([1 - 8e-17, 8.32667268e-17j]))
latex_string = sv.draw(output="latex_source")
assertTrue(latex_string.startswith(" |0\\rangle"))
assertNotIn("|1\\rangle", latex_string)

In [ ]:
"""Test state vector length"""
empty_vector = []
#with assertRaises(Exception):
#    empty_sv = AerStatevector([])
assertRaises(Exception, AerStatevector, [])